<h2>OBSERVATIONAL ASTROPHYSICS – FALL 2021 Reduction Exercise: Part 1</h2>

The goal of our reduction will be to complete the following steps:
1. make a master bias frame by combining all of our individual bias frames;
2. subtract this frame from all of the individual dark, science, and twilight flat frames;
3. combine your bias subtracted dark frames;
4. subtract the dark frame from each of your science and flat field images;
5. We may trim the images to focus on the centers where the flatfielding is good.
6. make a combined flatfield image in each band;
7. flatfield the individual flatfield images (as a check) and the science images;
8. shift and align all of your science images;
9. combine your science images to obtain a final combined science image.
10. establish the orientation of your images and where exactly they are pointing.

Once this is done we will move onto the next phase, where we measure the brightness of our objects.

In this exercise you will go through and start by viewing all of your dark, bias, and science frames to check for any issues.  You will also access the logs and you will each make your own lists of files of each type.  We will do this for the twilight flats at a later point as they were taken by Dr. Finn after we observed and I need to double check which ones we can use.

<h4> Copy over the images from a public shared drive to <i>your</i> departmental shared disk space </h4>

The department has created space for each of you on its network drive space.  You will store all of your data there so that you can work from any computer.

Text in quotes <> indicates a placeholder value that you will need to fill in.  For example \<KUID> should be your KU ID number.  

The raw data are all stored in the ~/RFSLAB/USER_DPT/_PUBLIC/ASTR596/Fall_2021/Data/2021-10-19/Raw/

Your personal network drive space is at ~/RFSLAB/USER_DPT/\<KUID>

To copy over the data from the public directory to your directory go to your network directory in the command line and:

1. make a directory tree called "ASTR596/Data/2021-10-19/Raw" and cd into that directory.  Note that you will need to make mutliple directories one after the other to make this tree.

2. from within that directory type (without the quotes)

<i>rsync -u -a -v ~/RFSLAB/USER_DPT/_PUBLIC/ASTR596/Fall_2021/Data/2021-10-19/Raw/ .</i>

where there is a space before the final "."

3. Make directories in your personal space called 

~/RFSLAB/USER_DPT/\<KUID>/ASTR596/Data/2021-10-19/Reduced and 

~/RFSLAB/USER_DPT/\<KUID>/ASTR596/Data/2021-10-19/Files 

4. You will need to make lists that contain the different kinds of images.  Open the log file on google docs for 2021-10-19, which is the one night on which we were able to get data. https://docs.google.com/spreadsheets/d/1MPkdAEUSHp2LNUNKNUv27fghJJIgEMv7SAFZ_clElAI/edit#gid=2042569792  

In [1]:
import numpy as np
from astropy import units as u
from astropy.nddata import CCDData
import ccdproc
from matplotlib import pyplot as plt
from ccdproc import Combiner
from astropy.io import ascii

5. Do this for the bias list and the darks that have 180s exposures, as that was the same as for our science frames.

We will also need to do this for the twilight flats in each filter using a list called "flatlist_< filtname>", where filename shoudl be *g* or *r*.  *We will come back to this later once we find out from Dr. Finn which twilight flats were actually good.*



In [2]:
# this is a list of all the bias image names 
# you will need to complete my list
#********************
biaslist = np.array(["b033.fit","b034.fit","b035.fit","b036.fit","b037.fit","b038.fit","b039.fit","b040.fit",
                    "b041.fit","b042.fit","b043.fit","b044.fit","b045.fit","b046.fit","b047.fit","b048.fit",
                    "b049.fit","b050.fit","b051.fit","b052.fit","b053.fit"])

In [3]:
#this is a list of all the dark images that were of the same exposure as our science images
# you will need to complete my list
#*************
darklist_180s = np.array(["d001.fit","d002.fit","d003.fit","d004.fit","d005.fit","d006.fit","d007.fit",
                         "d008.fit","d009.fit","d010.fit","d011.fit"])


In [4]:
#this is a list of all the flatfield images.  There may need to be more than one list.
####We will have to fill this in later once we clarify which flatfield files actually worked well.  
####You can ignore it for now
#*************
flatlist_g = np.array(["skyflat_g.001.fit","skyflat_g.002.fit","skyflat_g.003.fit","skyflat_g.004.fit",
                       "skyflat_g.005.fit","skyflat_g.006.fit","skyflat_g.007.fit"])
flatlist_r = np.array(["skyflat_r.001.fit","skyflat_r.002.fit","skyflat_r.003.fit","skyflat_r.004.fit",
                       "skyflat_r.005.fit","skyflat_r.006.fit","skyflat_r.007.fit","skyflat_r.008.fit",
                       "skyflat_r.009.fit","skyflat_r.010.fit","skyflat_r.011.fit","skyflat_r.012.fit",
                       "skyflat_r.013.fit","skyflat_r.014.fit"])

6. Repeat for the science frames, where this should only include the frames that you are using for science on your clusters.  These are highlighted in green in the log.  These should be called, e.g. "sciencelist_< filtname>_< cluster name>"

In [5]:
#this is a list of all the science images.  There may need to be more than one list
#*************
sciencelist_g_Markarian50 = np.array(["s009.fit","s010.fit","s011.fit","s012.fit","s013.fit","s014.fit"])
sciencelist_r_Markarian50 = np.array(["s015.fit","s016.fit","s017.fit","s018.fit","s019.fit","s020.fit"])

7. open ds9 and one after the other display the images in the lists above, which is fewer than the actual number of images you took in a night, and examine them to make sure that there is nothing wrong with them, e.g. satellite trails, very elongated stars.  Make a list that contains any bad images there.  *Ask me if you are unsure what a bad image is.*

Then use the code below to trim the bad images from each of your lists.

In [13]:
#define a function that takes an array of image names and an array of bad images and 
#returns a list of images that were not in the bad list
def imlist_clean(imlist, badlist):

    #updated versions of your image list that excludes these bad images
    #this intializes it
    goodimlist = np.array([]) 

    #this loops through every element in badlist
    for i in range(len(imlist)): 
    
        #see if that element of imlist exists in the list of bad images.
        #ibad is an array of indices of the array badlist that match the element of imlist
        ibad = np.where(badlist == imlist[i]) 

        #this is just to reformat ibad, which is output by where as a 2D array, where the 
        #first element is the one we want
        ibad = ibad[0]  

        #this tests if there was a match with badlist.  If there was not, then append
        #the good image names to the good image list
        if ibad.size==0: 
            goodimlist = np.append(goodimlist,imlist[i]) 

    #return something from the routine
    return(goodimlist)


In [14]:
#a list of bad images
#*************
badlist = np.array([])

#make a cleaned version of the biaslist
biaslist_good = imlist_clean(biaslist,badlist)
print("biaslist_good = ",biaslist_good)

biaslist_good =  ['b033.fit' 'b034.fit' 'b035.fit' 'b036.fit' 'b037.fit' 'b038.fit'
 'b039.fit' 'b040.fit' 'b041.fit' 'b042.fit' 'b043.fit' 'b044.fit'
 'b045.fit' 'b046.fit' 'b047.fit' 'b048.fit' 'b049.fit' 'b050.fit'
 'b051.fit' 'b052.fit' 'b053.fit']


/tmp/ipykernel_20860/2024549829.py:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  ibad = np.where(badlist == imlist[i])


In [20]:
#a list of bad images
#*************
badlist = np.array([])

#make a cleaned version of the darklist
darklist_good = imlist_clean(darklist_180s,badlist)
print("darklist_good = ",darklist_good)

darklist_good =  ['d001.fit' 'd002.fit' 'd003.fit' 'd004.fit' 'd005.fit' 'd006.fit'
 'd007.fit' 'd008.fit' 'd009.fit' 'd010.fit' 'd011.fit']


/tmp/ipykernel_20860/2024549829.py:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  ibad = np.where(badlist == imlist[i])


In [17]:
#a list of bad images
#*************
badlist = np.array([])

#make a cleaned version of the flatlist_g
flatlist_g_good = imlist_clean(flatlist_g,badlist)
print("flatlist_g_good = ",flatlist_g_good)

flatlist_g_good =  ['skyflat_g.001.fit' 'skyflat_g.002.fit' 'skyflat_g.003.fit'
 'skyflat_g.004.fit' 'skyflat_g.005.fit' 'skyflat_g.006.fit'
 'skyflat_g.007.fit']


/tmp/ipykernel_20860/2024549829.py:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  ibad = np.where(badlist == imlist[i])


In [18]:
#a list of bad images
#*************
badlist = np.array([])

#make a cleaned version of the flatlist_r
flatlist_r_good = imlist_clean(flatlist_r,badlist)
print("flatlist_r_good = ",flatlist_r_good)

flatlist_r_good =  ['skyflat_r.001.fit' 'skyflat_r.002.fit' 'skyflat_r.003.fit'
 'skyflat_r.004.fit' 'skyflat_r.005.fit' 'skyflat_r.006.fit'
 'skyflat_r.007.fit' 'skyflat_r.008.fit' 'skyflat_r.009.fit'
 'skyflat_r.010.fit' 'skyflat_r.011.fit' 'skyflat_r.012.fit'
 'skyflat_r.013.fit' 'skyflat_r.014.fit']


/tmp/ipykernel_20860/2024549829.py:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  ibad = np.where(badlist == imlist[i])


<h4> Make a combined bias frame </h4>
    
We will be reducing our data using the "ccdproc" package of reduction routines contained in astropy.
    
You will first read in bias images and combine them using the "Combiner" task described at https://ccdproc.readthedocs.io/en/latest/image_combination.html

Make sure to read the section on sigma-clipping, which describes how we generate masks for combining the images and how we eventually combine the images.  

As a first step you will need to take the good lists of objects from above in each category and make a file for each class of objects, e.g. bias, dark, science_\<band>, that contain all the corresponding file names, with one object per line.  You can name the lists what you'd like, e.g. biaslist, but you'll need to make sure that they are in the right places in the program later on.  They should all be put in your 

~/RFSLAB/USER_DPT/\<KUID>/ASTR596/Data/2021-10-19/Files 

directory:
1. biaslist: all of your good trimmed bias frames.  This will be used to make your master bias frame

These will be used later

2. darklist_\<exptime>: all of your dark images of a given exposure time (exptime)
2. flatlist_\<band>: all of the trimmed flatfields corresponding to a given band.  Ignore this for now, but we will need it later.
3. science_\<band>_\<target>: all of the science frames for each target in each band




The readme docs that describe this ccdproc routines are given at: https://ccdproc.readthedocs.io/en/latest/reduction_toolbox.html

Everywhere with a \***** you will need to change the code

In [24]:
#*******
#You will need to change these to the paths to your Raw and Reduced Data
rawpath = "/home/y731z863/RFSLAB/USER_DPT/y731z863/Raw/"
redpath = "/home/y731z863/RFSLAB/USER_DPT/y731z863/Reduced/"
filepath = "/home/y731z863/RFSLAB/USER_DPT/y731z863/Files/"

biaslist = "biaslist2.txt"
biaslistpath = filepath + biaslist


#initialize list of bias frames
bias_imlist = []
#this way of opening the file ensures that it closes after the loop is done.
with open(biaslistpath,'r') as fp:
    #read first line
    line = fp.readline()

    #build the combine list of all images
    #read every subsequent line
    while line:
        #this removes the trailing newline charactter and converts the output list to a scalar
        imname = line.split()
        imname = imname[0]
        
        #create the image name, including the path to the raw data
        imstr = rawpath + imname
        #read that into a CCDData object.  This allows you to specify a unit, which we indicate as "adu"
        im = CCDData.read(imstr,unit = "adu")

        #make a list of all the CCDData instances of each image
        bias_imlist = bias_imlist + [im]

        line = fp.readline()

#Combiner list of all bias images
bias_comb = Combiner(bias_imlist)

#now create masks with sigma clipping algorithm
#This creates a mask for each image that is the size of each image but which has 0 values
#everywhere except those pixels in excess of low_thresh and high_thresh sigma from the median.
#Those pixels get a value of 1
#*************
#given the number of pixels in your image how must you set high_thresh to avoid flagging more than
#one pixel because of expected statistical variations.  Assume the noise is distributed like a Gaussian.
bias_comb.sigma_clipping(low_thresh=3, high_thresh=3, func=np.ma.median)

#Use these masks to combine the images.  This now becomes your master bias
master_bias = bias_comb.average_combine()

#now write out the master bias
masterbiaspath = redpath + 'master_bias.fit'
master_bias.write(masterbiaspath,overwrite=True)


Open your master_bias.fit file with ds9.  In a markup cell below answer the following questions:
1. what is the typical number of counts in this image determined by moving your cursor around the image?

The typical number of counts in this image is about 70-80.

2. How does this compare with some of the individual bias frames?

They are very similar. Since it should be the average of each individual bias frame, and all the bias frames are similar, it makes sense that the combined bias is similar to others.

3. Comment on what the combined bias frame looks like?

The combined bias frame still have some white dashed lines shown up as each individual bias frame has. It looks basically identical to other individual bias frames. 